In [78]:
# !pip install pamda

In [79]:
# Finally I found a way to run fema_model on public device


# !pip install gdown
# folder_id = "1RMi5T81IxGm5d1h3qmzyTVWs906UXNwG"
# !gdown --folder https://drive.google.com/drive/folders/{folder_id}
# !pip install ./fema_model

In [80]:
# from google.colab import drive
# drive.mount('fema-fl-data-SR_Branch')

In [81]:
# !pip install /content/fema-fl-data-SR_Branch/MyDrive/GitHub/fema-fl-data-SR_Branch/fema_model

In [82]:
from pamda import pamda as p
from fema_model import Multi_Period_Terminal_System
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from tqdm import tqdm


In [83]:
geo_areas = p.read_json('geo_areas.json')

In [84]:
groups_df = pd.read_excel('NMSZ_Groups.xlsx')
groups = groups_df['group_name'].values
terminals_df = pd.read_excel('NMSZ_Terminals.xlsx')
stations_df = pd.read_excel('NMSZ_Stations.xlsx')

In [85]:
# Fixed parameters
# Bay queue space
b_q = 3
# Time to empty fuel at station (mins)
r_s = 60
# Bounds of time to empty fuel at station (mins)
r_s_bound = 10
# Truck start time max (hrs)
a = 6
# Truck capacity (gallons)
c = 9000
# Truck stopping criteria (hr)
tau = 0.5

# Flexible parameters
# Gate check-in time (mins)
# Baseline
# r_g = 7
# Disruption
# r_g = 10.5
# r_g = 8.4
# Intervention
r_g = 3.5
# Bounds multiplier of gate check-in time (mins)
r_g_bound = 0.1
# Bay filling time (mins)
# Baseline
# r_b = 35
# Disruption
# r_b = 52.5
# r_b = 42
# # Intervention
r_b = 17.5
# Bounds multiplier of bay filling time (mins)
r_b_bound = 0.1
# Truck speed (mph)
# Baseline
# v = 45
# Disruption
# v = 22.5
# v = 36
# Intervention
v = 67.5
# Bounds multiplier of truck speed (mph)
v_bound = 0.1
# Hours of service (hrs)
# Baseline
# h = 14
# HoS extension
h = 18

# Iterations
itrn = 50

# Final outputs for all groups
ops_df = pd.DataFrame(columns = ['Iteration', 'Group', 'Flow', 'Flow_MMgal', 'Trips_Per_Truck', 'Gate_Queue_Wait', 'Driving_Time'])

# Input kwargs
inputs = {
    'carry_over_demand_multiplier':0,
    'default_kwargs':{
        'demand_situation':'dem_scen_1',
        'demand_multiplier':1,
        'truck_multiplier':1,
        'station_algorithm':'max',
        'truck_kwargs':{
            'size':c,
            'speed':v,
            'speed_sigma':v*v_bound,
            'empty_rate':r_s,
            'empty_rate_sigma':r_s_bound,
            'open_time_min':0,
            'open_time_max':a,
            'close_time_after_open':h,
            'close_early_delta':tau
        },
        'station_kwargs':{
            'open_time':0,
            'close_time':24,
        },
        'terminal_kwargs':{
            'gate_kwargs':{
                'gate_rate':r_g,
                'gate_rate_sigma':r_g*r_g_bound,
                'open_time':0,
                'close_time':24,
                'extra_gates':0,
                'max_gate_queue':None, #Infinite
                'share_gate_queue_bool':True
            },
            'bay_kwargs':{
                'open_time':0,
                'close_time':24,
                'fill_rate':r_b,
                'fill_rate_sigma':r_b*r_b_bound,
                'extra_bays':0,
                'max_bay_queue':b_q,
                'share_bay_queue_bool':True
            }
        }
    }
}

# Selected terminal interventions
inputs['terminal_kwargs'] = {}

# Create terminal systems dictionary
terminal_system_data = {}
for i in range(len(groups)):
    terminal_system_data[groups[i]] = {}
    # Baseline
    terminal_system_data[groups[i]]['trucks_available'] = int(groups_df['trucks'][i])
    # Disruption
    # terminal_system_data[groups[i]]['trucks_available'] = int(np.ceil(groups_df['trucks'][i]*0.5))
    # terminal_system_data[groups[i]]['trucks_available'] = int(np.ceil(groups_df['trucks'][i]*0.8))
    # Fixed fleet intervention
    # terminal_system_data[groups[i]]['trucks_available'] = int(np.ceil(groups_df['trucks'][i]*0.5))+20
    # Fleet intervention
    # terminal_system_data[groups[i]]['trucks_available'] = int(np.ceil(groups_df['trucks'][i]*0.75))
    terms = terminals_df[terminals_df['group_name'] == groups[i]]['terminal_id'].values
    terminal_system_data[groups[i]]['terminals'] = {}
    for j in range(len(terms)):
        terminal_system_data[groups[i]]['terminals'][terms[j]] = {}
        terminal_system_data[groups[i]]['terminals'][terms[j]]['fuel_types'] = 'a'
        terminal_system_data[groups[i]]['terminals'][terms[j]]['num_gates'] = int(terminals_df[terminals_df['terminal_id'] == terms[j]]['num_gates'].values[0])
        terminal_system_data[groups[i]]['terminals'][terms[j]]['num_bays'] = int(terminals_df[terminals_df['terminal_id'] == terms[j]]['num_bays'].values[0])
        # Terminal interventions for 2 largest terminal in the group
        # if(j >= len(terms)-2):
        #     inputs['terminal_kwargs'][terms[j]] = {}
        #     inputs['terminal_kwargs'][terms[j]]['gate_kwargs'] = {}
        #     inputs['terminal_kwargs'][terms[j]]['gate_kwargs']['gate_rate'] = 3.5
        #     inputs['terminal_kwargs'][terms[j]]['bay_kwargs'] = {}
        #     inputs['terminal_kwargs'][terms[j]]['bay_kwargs']['fill_rate'] = 17.5
        # Terminal interventions for largest terminal in the group
        # if(j == len(terms)-1):
        #     inputs['terminal_kwargs'][terms[j]] = {}
        #     inputs['terminal_kwargs'][terms[j]]['gate_kwargs'] = {}
        #     inputs['terminal_kwargs'][terms[j]]['gate_kwargs']['gate_rate'] = 3.5
        #     inputs['terminal_kwargs'][terms[j]]['bay_kwargs'] = {}
        #     inputs['terminal_kwargs'][terms[j]]['bay_kwargs']['fill_rate'] = 17.5

for l in tqdm(range(itrn)):
    op_flow = np.zeros((len(groups),))
    op_trips = np.zeros((len(groups),))
    op_gate_wait = np.zeros((len(groups),))
    op_driving = np.zeros((len(groups),))
    for i in range(len(groups)):
        stn = stations_df[stations_df['group_name'] == groups[i]]
        stn = shuffle(stn).reset_index()
        terminal_system_data[groups[i]]['stations'] = {}
        for k in range(len(stn)):
            station_name = stn['station_id'][k]
            terminal_system_data[groups[i]]['stations'][station_name] = {}
            terminal_system_data[groups[i]]['stations'][station_name]['travel_distance'] = stn['station_group_dist'][k]
            terminal_system_data[groups[i]]['stations'][station_name]['geo_code'] = '1'
            terminal_system_data[groups[i]]['stations'][station_name]['demand_scenarios'] = {}
            terminal_system_data[groups[i]]['stations'][station_name]['demand_scenarios']['dem_scen_1'] = {}
            terminal_system_data[groups[i]]['stations'][station_name]['demand_scenarios']['dem_scen_1']['demand'] = c
            terminal_system_data[groups[i]]['stations'][station_name]['demand_scenarios']['dem_scen_1']['fuel_types'] = ['a']
    periods = [inputs]
    model = Multi_Period_Terminal_System(terminal_system_data = terminal_system_data, geo_areas = geo_areas, periods = periods)
    output = model.serialize(minify = True)
    for i in range(len(groups)):
        op_flow[i] = round((output['0']['terminal_groups'][groups[i]]['statistics']['sum_met_total_demand']),3)
        op_trips[i] = round((output['0']['terminal_groups'][groups[i]]['statistics']['truck_avg_deliveries_made']),3)
        op_gate_wait[i] = round((output['0']['terminal_groups'][groups[i]]['statistics']['trip_avg_in_gate_queue']),3)
        op_driving[i] = round((output['0']['terminal_groups'][groups[i]]['statistics']['truck_avg_to_station'] +\
                               output['0']['terminal_groups'][groups[i]]['statistics']['truck_avg_to_terminal_group']),3)
    op_df1 = pd.DataFrame(columns = ['Iteration', 'Group', 'Flow', 'Flow_MMgal', 'Trips_Per_Truck', 'Gate_Queue_Wait', 'Driving_Time'])

    op_df1['Flow'] = op_flow
    op_df1['Flow_MMgal'] = op_flow/(10**6)
    op_df1['Trips_Per_Truck'] = op_trips
    op_df1['Gate_Queue_Wait'] = op_gate_wait
    op_df1['Driving_Time'] = op_driving
    op_df1['Iteration'] = l
    op_df1['Group'] = groups
    ops_df = pd.concat([ops_df, op_df1], ignore_index=True)

# Baseline
ops_df.to_csv('2025-02-23_NMSZ_Mississippi_River_Results.csv')
# Gate intervention
# ops_df.to_csv('2024-12-25_NMSZ_No_Memphis_Results_Gate_intervention.csv')

# print("File saved as 2024-11-27_NMSZ_Baseline_Results.csv")
# Disruption
# ops_df.to_csv('2022-03-15_Florida_Disruption2_Results.csv')
# Disruption + HoS extension
# ops_df.to_csv('2022-03-15_Florida_Disruption2_HoS_Results.csv')
# Disruption + HoS extension + single terminal intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_HoS_1_Terminal_Results.csv')
# Disruption + HoS extension + two terminals intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_HoS_2_Terminals_Results.csv')
# Disruption + HoS extension + all terminals intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_HoS_Terminals_Results.csv')
# Disruption + HoS extension + speed intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_HoS_Speed_Results.csv')
# Disruption + HoS extension + fixed fleet intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_HoS_Fixed_Fleet_Results.csv')
# Disruption + HoS extension + fleet intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_HoS_Fleet_Results.csv')
# Disruption + HoS extension + speed + fleet intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_HoS_Speed_Fleet2_Results.csv')
# Disruption + all intervention
# ops_df.to_csv('2022-03-15_Florida_Disruption_All_Interventions2_Results.csv')

  0%|          | 0/50 [00:00<?, ?it/s]<ipython-input-85-0f4ee0bfc4ba>:178: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ops_df = pd.concat([ops_df, op_df1], ignore_index=True)
100%|██████████| 50/50 [04:24<00:00,  5.29s/it]


In [86]:
print("File saved as 2025-02-15_NMSZ_Mississippi_baseline_Results.csv")

File saved as 2025-02-15_NMSZ_Mississippi_baseline_Results.csv


In [87]:
ops_df

,Iteration,Group,Flow,Flow_MMgal,Trips_Per_Truck,Gate_Queue_Wait,Driving_Time
0,0,Cape Girardeau,1800000.0,1.800,4.878,0.031,6.422
1,0,Evansville,1719000.0,1.719,5.618,0.000,5.850
2,0,Greenville,1053000.0,1.053,4.680,0.002,7.300
3,0,Jonesboro,1512000.0,1.512,4.200,0.002,8.278
4,0,Little Rock,4995000.0,4.995,4.912,0.000,6.377
...,...,...,...,...,...,...,...
595,49,Paducah,1701000.0,1.701,4.295,0.031,7.405
596,49,Princeton,810000.0,0.810,6.000,0.017,6.112
597,49,Robinson,990000.0,0.990,4.783,0.001,7.582
598,49,St. Louis,6687000.0,6.687,3.135,0.740,2.302


In [88]:
# Group by 'Group' and describe aggregated statistics
grouped_description = ops_df.groupby('Group').describe()

# Display the result
print(grouped_description)


                Flow                                                 \
               count       mean           std        min        25%   
Group                                                                 
Cape Girardeau  50.0  1780920.0  23188.526368  1683000.0  1773000.0   
Evansville      50.0  1733580.0  37303.000919  1665000.0  1710000.0   
Greenville      50.0  1101240.0  31322.913306  1044000.0  1073250.0   
Jonesboro       50.0  1558620.0  34050.316909  1485000.0  1539000.0   
Little Rock     50.0  5123520.0  79673.630186  4932000.0  5067000.0   
Memphis         50.0  8128080.0  99999.151017  7911000.0  8064000.0   
Owensboro       50.0  1504620.0  41570.147256  1422000.0  1487250.0   
Paducah         50.0  1677780.0  38660.896238  1602000.0  1649250.0   
Princeton       50.0   774360.0  25582.806726   720000.0   756000.0   
Robinson        50.0   984420.0  34446.106975   909000.0   963000.0   
St. Louis       50.0  6687000.0      0.000000  6687000.0  6687000.0   
Wood R

In [89]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Average of 'Flow' for each group
flow_avg = ops_df.groupby('Group')['Flow'].mean()


ax = flow_avg.plot(kind='bar', figsize=(10, 6), title='Mississippi River Non-Steady-State OFC by Group Terminals', color='orange')
ax.set_ylabel('OFC (MMgal/day)')
ax.set_xlabel('Group Terminals')

# Custom tick formatting to display values in millions
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x / 1_000_000:,.1f}'))  # Convert to millions and format to 1 decimal

# Set Y-axis limits
ax.set_ylim(0, 4000000)  # Set maximum Y-axis value to 6.5

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
